# Chapter 11

In [1]:
%use s2

println("Chapter 11 demos")

Chapter 11 demos


In [2]:
println("Penalty method")

val f: RealScalarFunction = object : AbstractBivariateRealFunction() {
    override fun evaluate(x: Double, y: Double): Double {
        // f = (x+1)^2 + (y+1)^2
        return (x + 1) * (x + 1) + (y + 1) * (y + 1)
    }
}

val c1: RealScalarFunction = object : AbstractBivariateRealFunction() {
    override fun evaluate(x: Double, y: Double): Double {
        // y = 0
        return y
    }
}

val c2: RealScalarFunction = object : AbstractBivariateRealFunction() {
    override fun evaluate(x: Double, y: Double): Double {
        // x >= 1
        return 1 - x
    }
}

val problem: ConstrainedOptimProblemImpl1
        = ConstrainedOptimProblemImpl1(
                f,
                GeneralEqualityConstraints(c1), // y = 0
                GeneralLessThanConstraints(c2)) // x >= 1

val M: Double = 1e30 // the penalty factor
val optim: PenaltyMethodMinimizer
        = PenaltyMethodMinimizer(
                PenaltyMethodMinimizer.DEFAULT_PENALTY_FUNCTION_FACTORY,
                M,
                // the solver to solve the equivalent unconstrained optimization problem
                BFGSMinimizer(false, 1e-8, 200)
        )
val soln: IterativeSolution<Vector> = optim.solve(problem)

val xmin: Vector = soln.search( // the minimizer
        DenseVector(arrayOf(0.0, 0.0)) // an initial guess
)
val fxmin: Double = f.evaluate(xmin) // the mimimum
println(String.format("f(%s) = %f", xmin, fxmin))

// alternatively
println(String.format("f(%s) = %f", soln.minimizer(), soln.minimum()))

Penalty method
f([1.000000, 0.000000] ) = 5.000000
f([1.000000, 0.000000] ) = 5.000000


In [3]:
println("Genetic algorithm")

// the objective function to minimize
val f: RealScalarFunction = object : AbstractBivariateRealFunction() {
    override fun evaluate(x: Double, y: Double): Double {
        return x * x + y * y // x^2 + y^2
    }
}

// a uniform random number generator
val rng: RandomLongGenerator = UniformRNG()
rng.seed(123456798L)

// construct an instance of the genetic algorithm solver
val solver = SimpleGridMinimizer(
                // define the encodinng, crossover and mutation operator
                object : SimpleGridMinimizer.NewCellFactoryCtor {
                    override fun newCellFactory() : SimpleCellFactory {
                        return SimpleCellFactory(0.1, rng)
                    }
                },
                rng, // source of randomness for the GA
                1e-15, // a precision parameter
                500, // the maximum number of iterations
                500 // the maximum number of iterations of no improvement
        )

// run the solver to solve the optimization problem
val soln: IterativeSolution<Vector>
        = solver.solve(C2OptimProblemImpl(f))
val xmin: Vector = soln.search(
        // the boundaries: [-10, 10], [-10, 10]
    DenseVector(-10.0, 10.0),
    DenseVector(10.0, -10.0),
    DenseVector(10.0, 10.0),
    DenseVector(-10.0, -10.0)
)

val fxmin: Double = f.evaluate(xmin) // the mimimum
println(String.format("f(%s) = %f", xmin, fxmin))

Genetic algorithm
f([0.000000, 0.000000] ) = 0.000000


In [4]:
println("Differential Evolution")

// the objective function to minimize
val f: RealScalarFunction = object : AbstractBivariateRealFunction() {
    override fun evaluate(x: Double, y: Double): Double {
        return (x - 1) * (x - 1) + (y - 3) * (y - 3) // (x - 1)^2 + (y - 3)^2
    }
}

// construct an integer programming problem
val problem: IPProblem
        // both x and y need to be integers
        = IPProblemImpl1(f, null, null, intArrayOf(1, 2), 0.0)

// a uniform random number generator
val rng: RandomLongGenerator = UniformRNG()
rng.seed(123456798L)

// construct an instance of a genetic algorithm solver
val solver = DEOptim(
        object : DEOptim.NewCellFactory {
            override fun newCellFactory() : DEOptimCellFactory {
                return IntegralConstrainedCellFactory(
                        Rand1Bin(0.5, 0.5, rng), // the DE operator
                        IntegralConstrainedCellFactory.SomeIntegers(problem))
            }
        },
        rng, // a uniform random number generator
        1e-15, // a precision parameter
        100, // the maximum number of iterations
        20 // the maximum number of iterations of no improvement
)

val soln: IterativeSolution<Vector> = solver.solve(problem)
val xmin: Vector = soln.search(
        // the boundaries: [-10, 10], [-10, 10]
    DenseVector(-10.0, 10.0),
    DenseVector(10.0, -10.0),
    DenseVector(10.0, 10.0),
    DenseVector(-10.0, -10.0)
)

val fxmin: Double = f.evaluate(xmin) // the mimimum
println(String.format("f(%s) = %f", xmin, fxmin))

Differential Evolution
f([1.000000, 3.000000] ) = 0.000000


In [5]:
println("Simulated annealing")

// the objective function to minimize
val f: RealScalarFunction = object : AbstractRealScalarFunction(2) {

    override fun evaluate(x: Vector): Double {
        val x1: Double = x.get(1)
        val x2: Double = x.get(2)
        // (4 - 2.1*x(1)^2 + x(1)^4/3)*x(1)^2
        val term1: Double
                        = (4.0 - 2.1 * x1.pow(2.0) + x1.pow(4.0) / 3.0) * x1.pow(2.0)
        // x(1)*x(2)
        val term2: Double = x1 * x2
        // (-4 + 4*x(2)^2)*x(2)^2
        val term3: Double = (-4.0 + 4.0 * x2.pow(2.0)) * x2.pow(2.0)
        return term1 + term2 + term3
    }
}

// construct an optimization problem
val problem = object : OptimProblem {

    override fun f(): RealScalarFunction {
        return f
    }

    override fun dimension(): Int {
        return 2
    }
}

// stop after 5000 iterations
val stopCondition: StopCondition = AfterIterations(5000)
// an instance of a simulated annealing solver
val solver: IterativeMinimizer<OptimProblem> = GeneralizedSimulatedAnnealingMinimizer(
                2, // dimension of the objective function
                stopCondition
        )
val soln: IterativeSolution<Vector> = solver.solve(problem)
val x0: Vector = DenseVector(0.5, 0.5) // the initial guess
val xmin: Vector = soln.search(x0)
val fxmin: Double = f.evaluate(xmin) // the mimimum
println(String.format("f(%s) = %f", xmin, fxmin))

Simulated annealing
f([-0.089836, 0.712655] ) = -1.031628
